## Install the MMM package
- Use install.packages to install the final app package and load it to namespace 
- The dependent packages are automatically loaded
- The data objects are also loaded with the library function

In [1]:
install.packages('B2BAppV2_0.2.0.tar.gz',repos = NULL)
options(scipen = 10000)
set.seed(1234)

In [2]:
#library(tidyverse)
library(B2BAppV2)

Loading required package: tidyverse
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.3  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
Warning message:
"package 'purrr' was built under R version 3.6.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [3]:
dim(data_XyZ)

[1] 2610   77

In [4]:
data_XyZ %>% filter(qy == 'Q3_2019') %>% select(period) %>% distinct() %>% pull()

[1] "2019-06-30" "2019-07-07" "2019-07-14" "2019-07-21" "2019-07-28"
[6] "2019-08-04" "2019-08-11" "2019-08-18" "2019-08-25"

## Set up sample test inputs for both JSON and CSV
- Sample inputs to test the output from the run_mmm_app function 
- The csv output is used in the download data and the json is used for the UI
- Use the inputs in the run_mmm_app function to get the outputs
- The JSON output consists of two parts:
    - `summary` - Which outputs the summary of the Footprint
    - `spend` - Which outputs the DMA level spend by media
- The CSV outputs just the spend metrics by DMA and footprints

In [10]:
test_input_json = list('Input_OPT_budget' = 1500000,
'Input_SDL_budget' = 900000,
'Input_OPT_crosschannel_imp' = 50934471,
'Input_SDL_crosschannel_imp' = 7141942,
'Input_OPT_email_sent' = 59684,
'Input_SDL_email_sent' = 16852,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'json'                       
)

In [6]:
test_input_csv = list('Input_OPT_budget' = 1500000,
'Input_SDL_budget' = 900000,
'Input_OPT_crosschannel_imp' = 50934471,
'Input_SDL_crosschannel_imp' = 7141942,
'Input_OPT_email_sent' = 59684,
'Input_SDL_email_sent' = 16852,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

In [11]:
test_res_json <- run_b2b_app(test_input_json)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

In [12]:
test_res_json

$summary
  name Cross Channel Impressions Email Volume Gross Adds id __parent __color
1  USA                58,076,413       76,536     18,053  1   parent #000000
2 EAST                50,934,471       59,684     11,720  2        1 #0084d6
3 WEST                 7,141,942       16,852      6,333  3        1 #ff7f32
  expanded
1     true
2    false
3    false

$spend
  name Total Spend $(000) Digital DRTV Radio Search Social Direct Mail id
1  USA             $2,400    $748 $437  $409   $356   $257        $193  1
2 EAST             $1,500    $472 $206  $330   $296   $124         $71  2
3 WEST               $900    $276 $231   $79    $59   $133        $122  3
  __parent __color expanded
1   parent #000000     true
2        1 #0084d6    false
3        1 #ff7f32    false

In [13]:
test_res_csv <- run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

In [14]:
test_res_csv

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Search,Social,Direct Mail,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,400",$748,$437,$409,$356,$257,$193,"18,053","58,076,413","76,536"
EAST,"$1,500",$472,$206,$330,$296,$124,$71,"11,720","50,934,471","59,684"
WEST,$900,$276,$231,$79,$59,$133,$122,"6,333","7,141,942","16,852"


### SDL - testing

#### SDL - All ON

In [21]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 510000,to = 920000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q1',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [22]:
result_df

Footprint,Total Spend $(000) [Q1],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$510,$67,$154,$43,$127,$42,$78,"5,599","7,142",17
WEST,$520,$70,$156,$43,$130,$43,$79,"5,619","7,142",17
WEST,$530,$72,$158,$43,$134,$43,$80,"5,641","7,142",17
WEST,$540,$75,$161,$43,$138,$44,$80,"5,665","7,142",17
WEST,$550,$77,$164,$43,$141,$43,$81,"5,693","7,142",17
WEST,$560,$80,$168,$43,$145,$44,$81,"5,723","7,142",17
WEST,$570,$82,$169,$43,$149,$44,$83,"5,757","7,142",17
WEST,$580,$85,$172,$43,$152,$44,$85,"5,793","7,142",17
WEST,$590,$89,$171,$43,$157,$44,$86,"5,828","7,142",17
WEST,$600,$91,$172,$43,$161,$46,$88,"5,864","7,142",17


In [23]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 510000,to = 920000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q2',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [24]:
result_df

Footprint,Total Spend $(000) [Q2],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$510,$67,$154,$43,$127,$42,$78,"5,440","7,142",17
WEST,$520,$70,$156,$43,$130,$43,$79,"5,460","7,142",17
WEST,$530,$72,$158,$43,$134,$43,$80,"5,481","7,142",17
WEST,$540,$75,$161,$43,$138,$44,$80,"5,506","7,142",17
WEST,$550,$77,$164,$43,$141,$43,$81,"5,534","7,142",17
WEST,$560,$80,$168,$43,$145,$44,$81,"5,564","7,142",17
WEST,$570,$82,$169,$43,$149,$44,$83,"5,598","7,142",17
WEST,$580,$85,$172,$43,$152,$44,$85,"5,633","7,142",17
WEST,$590,$89,$171,$43,$157,$44,$86,"5,669","7,142",17
WEST,$600,$91,$172,$43,$161,$46,$88,"5,705","7,142",17


In [25]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 510000,to = 920000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q3',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [26]:
result_df

Footprint,Total Spend $(000) [Q3],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$510,$67,$154,$43,$127,$42,$78,"6,110","7,142",17
WEST,$520,$70,$156,$43,$130,$43,$79,"6,129","7,142",17
WEST,$530,$72,$158,$43,$134,$43,$80,"6,151","7,142",17
WEST,$540,$75,$161,$43,$138,$44,$80,"6,176","7,142",17
WEST,$550,$77,$164,$43,$141,$43,$81,"6,203","7,142",17
WEST,$560,$80,$168,$43,$145,$44,$81,"6,234","7,142",17
WEST,$570,$82,$169,$43,$149,$44,$83,"6,268","7,142",17
WEST,$580,$85,$172,$43,$152,$44,$85,"6,303","7,142",17
WEST,$590,$89,$171,$43,$157,$44,$86,"6,339","7,142",17
WEST,$600,$91,$172,$43,$161,$46,$88,"6,375","7,142",17


In [27]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 510000,to = 920000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [28]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$510,$67,$154,$43,$127,$42,$78,"5,195","7,142",17
WEST,$520,$70,$156,$43,$130,$43,$79,"5,214","7,142",17
WEST,$530,$72,$158,$43,$134,$43,$80,"5,236","7,142",17
WEST,$540,$75,$161,$43,$138,$44,$80,"5,261","7,142",17
WEST,$550,$77,$164,$43,$141,$43,$81,"5,288","7,142",17
WEST,$560,$80,$168,$43,$145,$44,$81,"5,319","7,142",17
WEST,$570,$82,$169,$43,$149,$44,$83,"5,353","7,142",17
WEST,$580,$85,$172,$43,$152,$44,$85,"5,388","7,142",17
WEST,$590,$89,$171,$43,$157,$44,$86,"5,424","7,142",17
WEST,$600,$91,$172,$43,$161,$46,$88,"5,460","7,142",17


#### SDL - DRTV & Radio - OFF

In [29]:
# multiple ... arguments
foreach(i=1:4, j=1:10) %do%
    sqrt(i+j)

ERROR: Error in foreach(i = 1:4, j = 1:10) %do% sqrt(i + j): could not find function "%do%"


In [30]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 350000,to = 620000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q1',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 0, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 0,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [31]:
result_df

Footprint,Total Spend $(000) [Q1],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$350,$74,$155,$0,$0,$42,$79,"5,501","7,142",17
WEST,$360,$78,$159,$0,$0,$43,$79,"5,508","7,142",17
WEST,$370,$83,$162,$0,$0,$43,$82,"5,515","7,142",17
WEST,$380,$90,$165,$0,$0,$43,$82,"5,525","7,142",17
WEST,$390,$94,$168,$0,$0,$43,$85,"5,535","7,142",17
WEST,$400,$99,$169,$0,$0,$45,$87,"5,545","7,142",17
WEST,$410,$103,$172,$0,$0,$46,$89,"5,556","7,142",17
WEST,$420,$105,$180,$0,$0,$45,$90,"5,567","7,142",17
WEST,$430,$108,$182,$0,$0,$47,$93,"5,580","7,142",17
WEST,$440,$111,$190,$0,$0,$47,$93,"5,587","7,142",17


In [32]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 350000,to = 620000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q2',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 0, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 0,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [33]:
result_df

Footprint,Total Spend $(000) [Q2],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$350,$74,$155,$0,$0,$42,$79,"5,342","7,142",17
WEST,$360,$78,$159,$0,$0,$43,$79,"5,349","7,142",17
WEST,$370,$83,$162,$0,$0,$43,$82,"5,356","7,142",17
WEST,$380,$90,$165,$0,$0,$43,$82,"5,366","7,142",17
WEST,$390,$94,$168,$0,$0,$43,$85,"5,376","7,142",17
WEST,$400,$99,$169,$0,$0,$45,$87,"5,386","7,142",17
WEST,$410,$103,$172,$0,$0,$46,$89,"5,397","7,142",17
WEST,$420,$105,$180,$0,$0,$45,$90,"5,408","7,142",17
WEST,$430,$108,$182,$0,$0,$47,$93,"5,421","7,142",17
WEST,$440,$111,$190,$0,$0,$47,$93,"5,428","7,142",17


In [34]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 350000,to = 620000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q3',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 0, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 0,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [35]:
result_df

Footprint,Total Spend $(000) [Q3],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$350,$74,$155,$0,$0,$42,$79,"6,012","7,142",17
WEST,$360,$78,$159,$0,$0,$43,$79,"6,019","7,142",17
WEST,$370,$83,$162,$0,$0,$43,$82,"6,025","7,142",17
WEST,$380,$90,$165,$0,$0,$43,$82,"6,036","7,142",17
WEST,$390,$94,$168,$0,$0,$43,$85,"6,045","7,142",17
WEST,$400,$99,$169,$0,$0,$45,$87,"6,056","7,142",17
WEST,$410,$103,$172,$0,$0,$46,$89,"6,067","7,142",17
WEST,$420,$105,$180,$0,$0,$45,$90,"6,078","7,142",17
WEST,$430,$108,$182,$0,$0,$47,$93,"6,090","7,142",17
WEST,$440,$111,$190,$0,$0,$47,$93,"6,098","7,142",17


In [36]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 350000,to = 620000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 0, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 0,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [37]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$350,$74,$155,$0,$0,$42,$79,"5,097","7,142",17
WEST,$360,$78,$159,$0,$0,$43,$79,"5,103","7,142",17
WEST,$370,$83,$162,$0,$0,$43,$82,"5,110","7,142",17
WEST,$380,$90,$165,$0,$0,$43,$82,"5,121","7,142",17
WEST,$390,$94,$168,$0,$0,$43,$85,"5,130","7,142",17
WEST,$400,$99,$169,$0,$0,$45,$87,"5,141","7,142",17
WEST,$410,$103,$172,$0,$0,$46,$89,"5,152","7,142",17
WEST,$420,$105,$180,$0,$0,$45,$90,"5,163","7,142",17
WEST,$430,$108,$182,$0,$0,$47,$93,"5,175","7,142",17
WEST,$440,$111,$190,$0,$0,$47,$93,"5,182","7,142",17


#### SDL - DRTV - OFF

In [38]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 390000,to = 700000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q1',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 0,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [39]:
result_df

Footprint,Total Spend $(000) [Q1],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$390,$72,$154,$44,$0,$42,$78,"5,502","7,142",17
WEST,$400,$76,$157,$45,$0,$42,$80,"5,508","7,142",17
WEST,$410,$82,$159,$47,$0,$43,$79,"5,516","7,142",17
WEST,$420,$86,$163,$48,$0,$43,$80,"5,524","7,142",17
WEST,$430,$91,$166,$49,$0,$43,$81,"5,534","7,142",17
WEST,$440,$95,$168,$48,$0,$44,$86,"5,545","7,142",17
WEST,$450,$97,$169,$51,$0,$45,$87,"5,556","7,142",17
WEST,$460,$97,$171,$56,$0,$46,$89,"5,568","7,142",17
WEST,$470,$99,$180,$54,$0,$45,$93,"5,576","7,142",17
WEST,$480,$98,$189,$58,$0,$47,$88,"5,584","7,142",17


In [40]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 390000,to = 700000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q2',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 0,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [41]:
result_df

Footprint,Total Spend $(000) [Q2],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$390,$72,$154,$44,$0,$42,$78,"5,343","7,142",17
WEST,$400,$76,$157,$45,$0,$42,$80,"5,349","7,142",17
WEST,$410,$82,$159,$47,$0,$43,$79,"5,357","7,142",17
WEST,$420,$86,$163,$48,$0,$43,$80,"5,365","7,142",17
WEST,$430,$91,$166,$49,$0,$43,$81,"5,375","7,142",17
WEST,$440,$95,$168,$48,$0,$44,$86,"5,386","7,142",17
WEST,$450,$97,$169,$51,$0,$45,$87,"5,397","7,142",17
WEST,$460,$97,$171,$56,$0,$46,$89,"5,409","7,142",17
WEST,$470,$99,$180,$54,$0,$45,$93,"5,417","7,142",17
WEST,$480,$98,$189,$58,$0,$47,$88,"5,425","7,142",17


In [42]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 390000,to = 700000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q3',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 0,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [43]:
result_df

Footprint,Total Spend $(000) [Q3],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$390,$72,$154,$44,$0,$42,$78,"6,012","7,142",17
WEST,$400,$76,$157,$45,$0,$42,$80,"6,019","7,142",17
WEST,$410,$82,$159,$47,$0,$43,$79,"6,026","7,142",17
WEST,$420,$86,$163,$48,$0,$43,$80,"6,034","7,142",17
WEST,$430,$91,$166,$49,$0,$43,$81,"6,044","7,142",17
WEST,$440,$95,$168,$48,$0,$44,$86,"6,055","7,142",17
WEST,$450,$97,$169,$51,$0,$45,$87,"6,066","7,142",17
WEST,$460,$97,$171,$56,$0,$46,$89,"6,078","7,142",17
WEST,$470,$99,$180,$54,$0,$45,$93,"6,087","7,142",17
WEST,$480,$98,$189,$58,$0,$47,$88,"6,095","7,142",17


In [44]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 390000,to = 700000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 0,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [45]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$390,$72,$154,$44,$0,$42,$78,"5,097","7,142",17
WEST,$400,$76,$157,$45,$0,$42,$80,"5,104","7,142",17
WEST,$410,$82,$159,$47,$0,$43,$79,"5,111","7,142",17
WEST,$420,$86,$163,$48,$0,$43,$80,"5,119","7,142",17
WEST,$430,$91,$166,$49,$0,$43,$81,"5,129","7,142",17
WEST,$440,$95,$168,$48,$0,$44,$86,"5,140","7,142",17
WEST,$450,$97,$169,$51,$0,$45,$87,"5,151","7,142",17
WEST,$460,$97,$171,$56,$0,$46,$89,"5,163","7,142",17
WEST,$470,$99,$180,$54,$0,$45,$93,"5,172","7,142",17
WEST,$480,$98,$189,$58,$0,$47,$88,"5,180","7,142",17


#### SDL - Radio - OFF

In [46]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 470000,to = 840000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q1',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 0, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [47]:
result_df

Footprint,Total Spend $(000) [Q1],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$470,$68,$155,$0,$127,$42,$78,"5,596","7,142",17
WEST,$480,$70,$157,$0,$131,$43,$79,"5,615","7,142",17
WEST,$490,$73,$160,$0,$134,$44,$80,"5,636","7,142",17
WEST,$500,$75,$163,$0,$137,$42,$83,"5,660","7,142",17
WEST,$510,$78,$168,$0,$141,$43,$81,"5,687","7,142",17
WEST,$520,$80,$169,$0,$144,$44,$83,"5,716","7,142",17
WEST,$530,$83,$172,$0,$147,$44,$85,"5,749","7,142",17
WEST,$540,$85,$174,$0,$151,$44,$86,"5,781","7,142",17
WEST,$550,$89,$172,$0,$155,$46,$88,"5,815","7,142",17
WEST,$560,$89,$178,$0,$159,$45,$89,"5,847","7,142",17


In [48]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 470000,to = 840000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q2',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 0, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [49]:
result_df

Footprint,Total Spend $(000) [Q2],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$470,$68,$155,$0,$127,$42,$78,"5,437","7,142",17
WEST,$480,$70,$157,$0,$131,$43,$79,"5,456","7,142",17
WEST,$490,$73,$160,$0,$134,$44,$80,"5,477","7,142",17
WEST,$500,$75,$163,$0,$137,$42,$83,"5,501","7,142",17
WEST,$510,$78,$168,$0,$141,$43,$81,"5,528","7,142",17
WEST,$520,$80,$169,$0,$144,$44,$83,"5,557","7,142",17
WEST,$530,$83,$172,$0,$147,$44,$85,"5,590","7,142",17
WEST,$540,$85,$174,$0,$151,$44,$86,"5,622","7,142",17
WEST,$550,$89,$172,$0,$155,$46,$88,"5,656","7,142",17
WEST,$560,$89,$178,$0,$159,$45,$89,"5,688","7,142",17


In [50]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 470000,to = 840000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q3',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 0, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [51]:
result_df

Footprint,Total Spend $(000) [Q3],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$470,$68,$155,$0,$127,$42,$78,"6,107","7,142",17
WEST,$480,$70,$157,$0,$131,$43,$79,"6,126","7,142",17
WEST,$490,$73,$160,$0,$134,$44,$80,"6,147","7,142",17
WEST,$500,$75,$163,$0,$137,$42,$83,"6,171","7,142",17
WEST,$510,$78,$168,$0,$141,$43,$81,"6,197","7,142",17
WEST,$520,$80,$169,$0,$144,$44,$83,"6,227","7,142",17
WEST,$530,$83,$172,$0,$147,$44,$85,"6,259","7,142",17
WEST,$540,$85,$174,$0,$151,$44,$86,"6,292","7,142",17
WEST,$550,$89,$172,$0,$155,$46,$88,"6,326","7,142",17
WEST,$560,$89,$178,$0,$159,$45,$89,"6,358","7,142",17


In [52]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

sdl_budgets <- seq(from = 470000,to = 840000, by = 10000)

for (SDL_budget in sdl_budgets){
    
    test_input_csv = list('Input_OPT_budget' = 1320000,
                            'Input_SDL_budget' = SDL_budget,                                  #changed
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 0, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}


Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [53]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$470,$68,$155,$0,$127,$42,$78,"5,192","7,142",17
WEST,$480,$70,$157,$0,$131,$43,$79,"5,210","7,142",17
WEST,$490,$73,$160,$0,$134,$44,$80,"5,232","7,142",17
WEST,$500,$75,$163,$0,$137,$42,$83,"5,256","7,142",17
WEST,$510,$78,$168,$0,$141,$43,$81,"5,282","7,142",17
WEST,$520,$80,$169,$0,$144,$44,$83,"5,312","7,142",17
WEST,$530,$83,$172,$0,$147,$44,$85,"5,344","7,142",17
WEST,$540,$85,$174,$0,$151,$44,$86,"5,377","7,142",17
WEST,$550,$89,$172,$0,$155,$46,$88,"5,411","7,142",17
WEST,$560,$89,$178,$0,$159,$45,$89,"5,443","7,142",17


### OPT - testing

#### OPT - All ON

In [11]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1300000,to = 2380000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q1',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coerci

In [12]:
result_df

Footprint,Total Spend $(000) [Q1],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,301",$71,$388,$216,$206,$296,$124,"11,079","50,934",60
EAST,"$1,320",$71,$395,$227,$206,$296,$124,"11,130","50,934",60
EAST,"$1,340",$71,$403,$239,$206,$296,$124,"11,185","50,934",60
EAST,"$1,360",$71,$412,$250,$206,$296,$124,"11,244","50,934",60
EAST,"$1,380",$71,$420,$262,$206,$296,$124,"11,305","50,934",60
EAST,"$1,400",$71,$429,$273,$206,$296,$124,"11,370","50,934",60
EAST,"$1,420",$71,$438,$284,$206,$296,$124,"11,437","50,934",60
EAST,"$1,440",$71,$446,$296,$206,$296,$124,"11,506","50,934",60
EAST,"$1,460",$71,$455,$307,$206,$296,$124,"11,576","50,934",60
EAST,"$1,480",$71,$463,$319,$206,$296,$124,"11,648","50,934",60


In [13]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1300000,to = 2380000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q2',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coerci

In [14]:
result_df

Footprint,Total Spend $(000) [Q2],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,301",$71,$388,$216,$206,$296,$124,"11,637","50,934",60
EAST,"$1,320",$71,$395,$227,$206,$296,$124,"11,688","50,934",60
EAST,"$1,340",$71,$403,$239,$206,$296,$124,"11,743","50,934",60
EAST,"$1,360",$71,$412,$250,$206,$296,$124,"11,801","50,934",60
EAST,"$1,380",$71,$420,$262,$206,$296,$124,"11,863","50,934",60
EAST,"$1,400",$71,$429,$273,$206,$296,$124,"11,927","50,934",60
EAST,"$1,420",$71,$438,$284,$206,$296,$124,"11,994","50,934",60
EAST,"$1,440",$71,$446,$296,$206,$296,$124,"12,063","50,934",60
EAST,"$1,460",$71,$455,$307,$206,$296,$124,"12,133","50,934",60
EAST,"$1,480",$71,$463,$319,$206,$296,$124,"12,205","50,934",60


In [15]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1300000,to = 2380000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q3',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coerci

In [16]:
result_df

Footprint,Total Spend $(000) [Q3],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,301",$71,$388,$216,$206,$296,$124,"10,844","50,934",60
EAST,"$1,320",$71,$395,$227,$206,$296,$124,"10,895","50,934",60
EAST,"$1,340",$71,$403,$239,$206,$296,$124,"10,950","50,934",60
EAST,"$1,360",$71,$412,$250,$206,$296,$124,"11,008","50,934",60
EAST,"$1,380",$71,$420,$262,$206,$296,$124,"11,070","50,934",60
EAST,"$1,400",$71,$429,$273,$206,$296,$124,"11,134","50,934",60
EAST,"$1,420",$71,$438,$284,$206,$296,$124,"11,201","50,934",60
EAST,"$1,440",$71,$446,$296,$206,$296,$124,"11,270","50,934",60
EAST,"$1,460",$71,$455,$307,$206,$296,$124,"11,341","50,934",60
EAST,"$1,480",$71,$463,$319,$206,$296,$124,"11,412","50,934",60


In [19]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1300000,to = 2380000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coerci

In [20]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,301",$71,$388,$216,$206,$296,$124,"10,538","50,934",60
EAST,"$1,320",$71,$395,$227,$206,$296,$124,"10,590","50,934",60
EAST,"$1,340",$71,$403,$239,$206,$296,$124,"10,644","50,934",60
EAST,"$1,360",$71,$412,$250,$206,$296,$124,"10,703","50,934",60
EAST,"$1,380",$71,$420,$262,$206,$296,$124,"10,764","50,934",60
EAST,"$1,400",$71,$429,$273,$206,$296,$124,"10,829","50,934",60
EAST,"$1,420",$71,$438,$284,$206,$296,$124,"10,896","50,934",60
EAST,"$1,440",$71,$446,$296,$206,$296,$124,"10,965","50,934",60
EAST,"$1,460",$71,$455,$307,$206,$296,$124,"11,035","50,934",60
EAST,"$1,480",$71,$463,$319,$206,$296,$124,"11,107","50,934",60


#### OPT - DRTV & Radio - OFF

In [5]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 900000,to = 1620000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q1',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 0,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [6]:
result_df

Footprint,Total Spend $(000) [Q1],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,$900,$71,$408,$0,$0,$296,$124,"10,580","50,934",60
EAST,$920,$71,$428,$0,$0,$296,$124,"10,590","50,934",60
EAST,$940,$71,$448,$0,$0,$296,$124,"10,602","50,934",60
EAST,$960,$71,$468,$0,$0,$296,$124,"10,615","50,934",60
EAST,$980,$71,$488,$0,$0,$296,$124,"10,629","50,934",60
EAST,"$1,000",$71,$508,$0,$0,$296,$124,"10,644","50,934",60
EAST,"$1,020",$71,$528,$0,$0,$296,$124,"10,661","50,934",60
EAST,"$1,040",$71,$548,$0,$0,$296,$124,"10,680","50,934",60
EAST,"$1,060",$71,$568,$0,$0,$296,$124,"10,700","50,934",60
EAST,"$1,080",$71,$588,$0,$0,$296,$124,"10,721","50,934",60


In [7]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 900000,to = 1620000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q2',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 0,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [8]:
result_df

Footprint,Total Spend $(000) [Q2],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,$900,$71,$408,$0,$0,$296,$124,"11,137","50,934",60
EAST,$920,$71,$428,$0,$0,$296,$124,"11,148","50,934",60
EAST,$940,$71,$448,$0,$0,$296,$124,"11,159","50,934",60
EAST,$960,$71,$468,$0,$0,$296,$124,"11,172","50,934",60
EAST,$980,$71,$488,$0,$0,$296,$124,"11,186","50,934",60
EAST,"$1,000",$71,$508,$0,$0,$296,$124,"11,202","50,934",60
EAST,"$1,020",$71,$528,$0,$0,$296,$124,"11,219","50,934",60
EAST,"$1,040",$71,$548,$0,$0,$296,$124,"11,237","50,934",60
EAST,"$1,060",$71,$568,$0,$0,$296,$124,"11,257","50,934",60
EAST,"$1,080",$71,$588,$0,$0,$296,$124,"11,278","50,934",60


In [9]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 900000,to = 1620000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q3',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 0,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [10]:
result_df

Footprint,Total Spend $(000) [Q3],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,$900,$71,$408,$0,$0,$296,$124,"10,344","50,934",60
EAST,$920,$71,$428,$0,$0,$296,$124,"10,355","50,934",60
EAST,$940,$71,$448,$0,$0,$296,$124,"10,366","50,934",60
EAST,$960,$71,$468,$0,$0,$296,$124,"10,379","50,934",60
EAST,$980,$71,$488,$0,$0,$296,$124,"10,393","50,934",60
EAST,"$1,000",$71,$508,$0,$0,$296,$124,"10,409","50,934",60
EAST,"$1,020",$71,$528,$0,$0,$296,$124,"10,426","50,934",60
EAST,"$1,040",$71,$548,$0,$0,$296,$124,"10,444","50,934",60
EAST,"$1,060",$71,$568,$0,$0,$296,$124,"10,464","50,934",60
EAST,"$1,080",$71,$588,$0,$0,$296,$124,"10,486","50,934",60


In [11]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 900000,to = 1620000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 0,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [12]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,$900,$71,$408,$0,$0,$296,$124,"10,039","50,934",60
EAST,$920,$71,$428,$0,$0,$296,$124,"10,049","50,934",60
EAST,$940,$71,$448,$0,$0,$296,$124,"10,061","50,934",60
EAST,$960,$71,$468,$0,$0,$296,$124,"10,074","50,934",60
EAST,$980,$71,$488,$0,$0,$296,$124,"10,088","50,934",60
EAST,"$1,000",$71,$508,$0,$0,$296,$124,"10,103","50,934",60
EAST,"$1,020",$71,$528,$0,$0,$296,$124,"10,120","50,934",60
EAST,"$1,040",$71,$548,$0,$0,$296,$124,"10,139","50,934",60
EAST,"$1,060",$71,$568,$0,$0,$296,$124,"10,159","50,934",60
EAST,"$1,080",$71,$588,$0,$0,$296,$124,"10,180","50,934",60


#### OPT - DRTV - OFF

In [29]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1100000,to = 2000000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q1',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 0,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [30]:
result_df

Footprint,Total Spend $(000) [Q1],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$392,$217,$0,$296,$124,"10,926","50,934",60
EAST,"$1,120",$71,$402,$226,$0,$296,$124,"10,969","50,934",60
EAST,"$1,140",$71,$412,$236,$0,$296,$124,"11,016","50,934",60
EAST,"$1,160",$71,$422,$246,$0,$296,$124,"11,065","50,934",60
EAST,"$1,180",$71,$433,$256,$0,$296,$124,"11,116","50,934",60
EAST,"$1,200",$71,$443,$265,$0,$296,$124,"11,170","50,934",60
EAST,"$1,220",$71,$453,$275,$0,$296,$124,"11,225","50,934",60
EAST,"$1,240",$71,$463,$285,$0,$296,$124,"11,283","50,934",60
EAST,"$1,260",$71,$474,$295,$0,$296,$124,"11,342","50,934",60
EAST,"$1,280",$71,$484,$304,$0,$296,$124,"11,402","50,934",60


In [31]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1100000,to = 2000000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q2',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 0,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [32]:
result_df

Footprint,Total Spend $(000) [Q2],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$392,$217,$0,$296,$124,"11,483","50,934",60
EAST,"$1,120",$71,$402,$226,$0,$296,$124,"11,527","50,934",60
EAST,"$1,140",$71,$412,$236,$0,$296,$124,"11,573","50,934",60
EAST,"$1,160",$71,$422,$246,$0,$296,$124,"11,622","50,934",60
EAST,"$1,180",$71,$433,$256,$0,$296,$124,"11,673","50,934",60
EAST,"$1,200",$71,$443,$265,$0,$296,$124,"11,727","50,934",60
EAST,"$1,220",$71,$453,$275,$0,$296,$124,"11,783","50,934",60
EAST,"$1,240",$71,$463,$285,$0,$296,$124,"11,840","50,934",60
EAST,"$1,260",$71,$474,$295,$0,$296,$124,"11,899","50,934",60
EAST,"$1,280",$71,$484,$304,$0,$296,$124,"11,959","50,934",60


In [33]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1100000,to = 2000000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q3',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 0,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [34]:
result_df

Footprint,Total Spend $(000) [Q3],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$392,$217,$0,$296,$124,"10,690","50,934",60
EAST,"$1,120",$71,$402,$226,$0,$296,$124,"10,734","50,934",60
EAST,"$1,140",$71,$412,$236,$0,$296,$124,"10,780","50,934",60
EAST,"$1,160",$71,$422,$246,$0,$296,$124,"10,829","50,934",60
EAST,"$1,180",$71,$433,$256,$0,$296,$124,"10,881","50,934",60
EAST,"$1,200",$71,$443,$265,$0,$296,$124,"10,934","50,934",60
EAST,"$1,220",$71,$453,$275,$0,$296,$124,"10,990","50,934",60
EAST,"$1,240",$71,$463,$285,$0,$296,$124,"11,047","50,934",60
EAST,"$1,260",$71,$474,$295,$0,$296,$124,"11,106","50,934",60
EAST,"$1,280",$71,$484,$304,$0,$296,$124,"11,166","50,934",60


In [35]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1100000,to = 2000000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 1,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 0,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [36]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$392,$217,$0,$296,$124,"10,385","50,934",60
EAST,"$1,120",$71,$402,$226,$0,$296,$124,"10,428","50,934",60
EAST,"$1,140",$71,$412,$236,$0,$296,$124,"10,475","50,934",60
EAST,"$1,160",$71,$422,$246,$0,$296,$124,"10,524","50,934",60
EAST,"$1,180",$71,$433,$256,$0,$296,$124,"10,575","50,934",60
EAST,"$1,200",$71,$443,$265,$0,$296,$124,"10,629","50,934",60
EAST,"$1,220",$71,$453,$275,$0,$296,$124,"10,685","50,934",60
EAST,"$1,240",$71,$463,$285,$0,$296,$124,"10,742","50,934",60
EAST,"$1,260",$71,$474,$295,$0,$296,$124,"10,801","50,934",60
EAST,"$1,280",$71,$484,$304,$0,$296,$124,"10,861","50,934",60


#### OPT - Radio - OFF

In [37]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1100000,to = 2000000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q1',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [38]:
result_df

Footprint,Total Spend $(000) [Q1],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"10,671","50,934",60
EAST,"$1,120",$71,$400,$0,$228,$296,$124,"10,686","50,934",60
EAST,"$1,140",$71,$409,$0,$239,$296,$124,"10,703","50,934",60
EAST,"$1,160",$71,$419,$0,$250,$296,$124,"10,722","50,934",60
EAST,"$1,180",$71,$428,$0,$261,$296,$124,"10,742","50,934",60
EAST,"$1,200",$71,$437,$0,$272,$296,$124,"10,764","50,934",60
EAST,"$1,220",$71,$446,$0,$282,$296,$124,"10,788","50,934",60
EAST,"$1,240",$71,$455,$0,$293,$296,$124,"10,813","50,934",60
EAST,"$1,260",$71,$464,$0,$304,$296,$124,"10,840","50,934",60
EAST,"$1,280",$71,$473,$0,$315,$296,$124,"10,868","50,934",60


In [39]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1100000,to = 2000000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q2',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [40]:
result_df

Footprint,Total Spend $(000) [Q2],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"11,228","50,934",60
EAST,"$1,120",$71,$400,$0,$228,$296,$124,"11,243","50,934",60
EAST,"$1,140",$71,$409,$0,$239,$296,$124,"11,260","50,934",60
EAST,"$1,160",$71,$419,$0,$250,$296,$124,"11,279","50,934",60
EAST,"$1,180",$71,$428,$0,$261,$296,$124,"11,299","50,934",60
EAST,"$1,200",$71,$437,$0,$272,$296,$124,"11,321","50,934",60
EAST,"$1,220",$71,$446,$0,$282,$296,$124,"11,345","50,934",60
EAST,"$1,240",$71,$455,$0,$293,$296,$124,"11,370","50,934",60
EAST,"$1,260",$71,$464,$0,$304,$296,$124,"11,397","50,934",60
EAST,"$1,280",$71,$473,$0,$315,$296,$124,"11,425","50,934",60


In [41]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1100000,to = 2000000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q3',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [42]:
result_df

Footprint,Total Spend $(000) [Q3],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"10,435","50,934",60
EAST,"$1,120",$71,$400,$0,$228,$296,$124,"10,451","50,934",60
EAST,"$1,140",$71,$409,$0,$239,$296,$124,"10,468","50,934",60
EAST,"$1,160",$71,$419,$0,$250,$296,$124,"10,486","50,934",60
EAST,"$1,180",$71,$428,$0,$261,$296,$124,"10,507","50,934",60
EAST,"$1,200",$71,$437,$0,$272,$296,$124,"10,529","50,934",60
EAST,"$1,220",$71,$446,$0,$282,$296,$124,"10,552","50,934",60
EAST,"$1,240",$71,$455,$0,$293,$296,$124,"10,577","50,934",60
EAST,"$1,260",$71,$464,$0,$304,$296,$124,"10,604","50,934",60
EAST,"$1,280",$71,$473,$0,$315,$296,$124,"10,633","50,934",60


In [43]:
result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

opt_budgets <- seq(from = 1100000,to = 2000000, by = 20000)

for (opt_budget in opt_budgets){
    
    test_input_csv = list('Input_OPT_budget' = opt_budget,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 50934471,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [44]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"10,130","50,934",60
EAST,"$1,120",$71,$400,$0,$228,$296,$124,"10,145","50,934",60
EAST,"$1,140",$71,$409,$0,$239,$296,$124,"10,162","50,934",60
EAST,"$1,160",$71,$419,$0,$250,$296,$124,"10,181","50,934",60
EAST,"$1,180",$71,$428,$0,$261,$296,$124,"10,201","50,934",60
EAST,"$1,200",$71,$437,$0,$272,$296,$124,"10,223","50,934",60
EAST,"$1,220",$71,$446,$0,$282,$296,$124,"10,247","50,934",60
EAST,"$1,240",$71,$455,$0,$293,$296,$124,"10,272","50,934",60
EAST,"$1,260",$71,$464,$0,$304,$296,$124,"10,299","50,934",60
EAST,"$1,280",$71,$473,$0,$315,$296,$124,"10,327","50,934",60


#### Cross channel

In [13]:
### OPT

result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

cc_imp <- seq(from = 5010000,to = 9270000, by = 15000)

for (cc in cc_imp){
    
    test_input_csv = list('Input_OPT_budget' = 1100000,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = cc,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coerci

Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and charact

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

...

Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and charact

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coerci

Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and charact

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [15]:
write.csv(result_df,'cc_imp_optimum.csv')

In [16]:
### SDL

result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

cc_imp <- seq(from = 35700000,to = 66200000, by = 100000)

for (cc in cc_imp){
    
    test_input_csv = list('Input_OPT_budget' = 1100000,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 5010000,
                            'Input_SDL_crosschannel_imp' = cc,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coerci

Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and charact

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

...

Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and charact

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coerci

Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and charact

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [17]:
write.csv(result_df,'cc_imp_SDL.csv')

#### Email

In [18]:
### OPT

result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

email_vol <- seq(from = 42000,to = 77000, by = 1000)

for (email in email_vol){
    
    test_input_csv = list('Input_OPT_budget' = 1100000,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 5010000,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = email,
                            'Input_SDL_email_sent' = 16852,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'EAST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [19]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,707","5,010",42
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,708","5,010",43
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,709","5,010",44
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,710","5,010",45
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,711","5,010",46
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,713","5,010",47
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,714","5,010",48
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,715","5,010",49
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,717","5,010",50
EAST,"$1,100",$71,$391,$0,$217,$296,$124,"9,718","5,010",51


In [20]:
### SDL

result_df <- data.frame(matrix(ncol = 11, nrow = 0))
x <- c('Footprint', 'Total Spend $(000) [Q1]', 'Direct Mail', 'Digital', 'Radio', 'DRTV', 'Search', 'Social', 'Gross Adds', 'Cross Channel Impressions (000)', 'Email Volume (000)')
colnames(result_df) <- x

email_vol <- seq(from = 12000,to = 21750, by = 250)

for (email in email_vol){
    
    test_input_csv = list('Input_OPT_budget' = 1100000,
                            'Input_SDL_budget' = 510000,                                 
                            'Input_OPT_crosschannel_imp' = 5010000,
                            'Input_SDL_crosschannel_imp' = 7141942,
                            'Input_OPT_email_sent' = 59684,
                            'Input_SDL_email_sent' = email,                       
                            'input_period' = 'Q4',
                            'flight_flag_radio_OPT' = 0,
                            'flight_flag_radio_SDL' = 1, 
                            'flight_flag_DRTV_OPT' = 1,
                            'flight_flag_DRTV_SDL' = 1,
                            'output_type' = 'csv' 
                           )

    df <- run_b2b_app(test_input_csv)
    df1 <- df %>% filter(Footprint == 'WEST') 
    
    result_df <- rbind(result_df, df1)
    
}

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vec

"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"Warning message:
"Column `dma` joi

In [21]:
result_df

Footprint,Total Spend $(000) [Q4],Direct Mail,Digital,Radio,DRTV,Search,Social,Gross Adds,Cross Channel Impressions (000),Email Volume (000)
WEST,$510,$67,$154,$43,$127,$42,$78,"5,189","7,142",12
WEST,$510,$67,$154,$43,$127,$42,$78,"5,190","7,142",12
WEST,$510,$67,$154,$43,$127,$42,$78,"5,190","7,142",12
WEST,$510,$67,$154,$43,$127,$42,$78,"5,190","7,142",13
WEST,$510,$67,$154,$43,$127,$42,$78,"5,190","7,142",13
WEST,$510,$67,$154,$43,$127,$42,$78,"5,190","7,142",13
WEST,$510,$67,$154,$43,$127,$42,$78,"5,191","7,142",14
WEST,$510,$67,$154,$43,$127,$42,$78,"5,191","7,142",14
WEST,$510,$67,$154,$43,$127,$42,$78,"5,191","7,142",14
WEST,$510,$67,$154,$43,$127,$42,$78,"5,191","7,142",14


## Test cases for deployment - given to Varun

### All ON

#### Q1

In [11]:
test_input_csv = list('Input_OPT_budget' = 1320000,
'Input_SDL_budget' = 510000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,830",$549,$333,$270,$138,$338,$202,"16,412","45,010,000","65,000"
EAST,"$1,320",$395,$206,$227,$71,$296,$124,"11,009","40,000,000","50,000"
WEST,$510,$154,$127,$43,$67,$42,$78,"5,403","5,010,000","15,000"


In [12]:
test_input_csv = list('Input_OPT_budget' = 1340000,
'Input_SDL_budget' = 520000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,860",$560,$336,$281,$141,$339,$203,"16,540","50,040,000","67,000"
EAST,"$1,340",$403,$206,$239,$71,$296,$124,"11,113","45,000,000","51,000"
WEST,$520,$156,$130,$43,$70,$43,$79,"5,427","5,040,000","16,000"


In [13]:
test_input_csv = list('Input_OPT_budget' = 2380000,
'Input_SDL_budget' = 540000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,920",$880,$520,$430,$207,$594,$287,"18,880","50,040,000","71,000"
EAST,"$2,380",$720,$383,$388,$132,$550,$207,"13,403","45,000,000","53,000"
WEST,$540,$161,$138,$43,$75,$44,$80,"5,477","5,040,000","18,000"


#### Q2

In [14]:
test_input_csv = list('Input_OPT_budget' = 1320000,
'Input_SDL_budget' = 510000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,830",$549,$333,$270,$138,$338,$202,"16,810","45,010,000","65,000"
EAST,"$1,320",$395,$206,$227,$71,$296,$124,"11,566","40,000,000","50,000"
WEST,$510,$154,$127,$43,$67,$42,$78,"5,244","5,010,000","15,000"


In [15]:
test_input_csv = list('Input_OPT_budget' = 1340000,
'Input_SDL_budget' = 520000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,860",$560,$336,$281,$141,$339,$203,"16,938","50,040,000","67,000"
EAST,"$1,340",$403,$206,$239,$71,$296,$124,"11,671","45,000,000","51,000"
WEST,$520,$156,$130,$43,$70,$43,$79,"5,268","5,040,000","16,000"


In [16]:
test_input_csv = list('Input_OPT_budget' = 2380000,
'Input_SDL_budget' = 540000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,920",$880,$520,$430,$207,$594,$287,"19,278","50,040,000","71,000"
EAST,"$2,380",$720,$383,$388,$132,$550,$207,"13,961","45,000,000","53,000"
WEST,$540,$161,$138,$43,$75,$44,$80,"5,317","5,040,000","18,000"


#### Q3

In [17]:
test_input_csv = list('Input_OPT_budget' = 1320000,
'Input_SDL_budget' = 510000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,830",$549,$333,$270,$138,$338,$202,"16,687","45,010,000","65,000"
EAST,"$1,320",$395,$206,$227,$71,$296,$124,"10,773","40,000,000","50,000"
WEST,$510,$154,$127,$43,$67,$42,$78,"5,914","5,010,000","15,000"


In [18]:
test_input_csv = list('Input_OPT_budget' = 1340000,
'Input_SDL_budget' = 520000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,860",$560,$336,$281,$141,$339,$203,"16,815","50,040,000","67,000"
EAST,"$1,340",$403,$206,$239,$71,$296,$124,"10,878","45,000,000","51,000"
WEST,$520,$156,$130,$43,$70,$43,$79,"5,937","5,040,000","16,000"


In [19]:
test_input_csv = list('Input_OPT_budget' = 2380000,
'Input_SDL_budget' = 540000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,920",$880,$520,$430,$207,$594,$287,"19,155","50,040,000","71,000"
EAST,"$2,380",$720,$383,$388,$132,$550,$207,"13,168","45,000,000","53,000"
WEST,$540,$161,$138,$43,$75,$44,$80,"5,987","5,040,000","18,000"


#### Q4

In [20]:
test_input_csv = list('Input_OPT_budget' = 1320000,
'Input_SDL_budget' = 510000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,830",$549,$333,$270,$138,$338,$202,"15,467","45,010,000","65,000"
EAST,"$1,320",$395,$206,$227,$71,$296,$124,"10,468","40,000,000","50,000"
WEST,$510,$154,$127,$43,$67,$42,$78,"4,999","5,010,000","15,000"


In [21]:
test_input_csv = list('Input_OPT_budget' = 1340000,
'Input_SDL_budget' = 520000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,860",$560,$336,$281,$141,$339,$203,"15,595","50,040,000","67,000"
EAST,"$1,340",$403,$206,$239,$71,$296,$124,"10,572","45,000,000","51,000"
WEST,$520,$156,$130,$43,$70,$43,$79,"5,022","5,040,000","16,000"


In [22]:
test_input_csv = list('Input_OPT_budget' = 2380000,
'Input_SDL_budget' = 540000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,920",$880,$520,$430,$207,$594,$287,"17,934","50,040,000","71,000"
EAST,"$2,380",$720,$383,$388,$132,$550,$207,"12,863","45,000,000","53,000"
WEST,$540,$161,$138,$43,$75,$44,$80,"5,072","5,040,000","18,000"


### DRTV & Radio - OFF

#### Q1

In [23]:
test_input_csv = list('Input_OPT_budget' = 900000,
'Input_SDL_budget' = 350000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,250",$564,$0,$0,$145,$339,$203,"15,764","45,010,000","65,000"
EAST,$900,$408,$0,$0,$71,$296,$124,"10,458","40,000,000","50,000"
WEST,$350,$155,$0,$0,$74,$42,$79,"5,305","5,010,000","15,000"


In [24]:
test_input_csv = list('Input_OPT_budget' = 920000,
'Input_SDL_budget' = 360000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,280",$588,$0,$0,$149,$340,$203,"15,834","50,040,000","67,000"
EAST,$920,$428,$0,$0,$71,$296,$124,"10,518","45,000,000","51,000"
WEST,$360,$159,$0,$0,$78,$43,$79,"5,316","5,040,000","16,000"


In [25]:
test_input_csv = list('Input_OPT_budget' = 1620000,
'Input_SDL_budget' = 620000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,240","$1,002",$0,$0,$254,$625,$359,"16,528","50,040,000","71,000"
EAST,"$1,620",$720,$0,$0,$132,$550,$218,"11,033","45,000,000","53,000"
WEST,$620,$282,$0,$0,$122,$74,$142,"5,495","5,040,000","18,000"


#### Q2

In [26]:
test_input_csv = list('Input_OPT_budget' = 900000,
'Input_SDL_budget' = 350000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,250",$564,$0,$0,$145,$339,$203,"16,162","45,010,000","65,000"
EAST,$900,$408,$0,$0,$71,$296,$124,"11,015","40,000,000","50,000"
WEST,$350,$155,$0,$0,$74,$42,$79,"5,146","5,010,000","15,000"


In [27]:
test_input_csv = list('Input_OPT_budget' = 920000,
'Input_SDL_budget' = 360000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,280",$588,$0,$0,$149,$340,$203,"16,232","50,040,000","67,000"
EAST,$920,$428,$0,$0,$71,$296,$124,"11,076","45,000,000","51,000"
WEST,$360,$159,$0,$0,$78,$43,$79,"5,157","5,040,000","16,000"


In [28]:
test_input_csv = list('Input_OPT_budget' = 1620000,
'Input_SDL_budget' = 620000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,240","$1,002",$0,$0,$254,$625,$359,"16,926","50,040,000","71,000"
EAST,"$1,620",$720,$0,$0,$132,$550,$218,"11,590","45,000,000","53,000"
WEST,$620,$282,$0,$0,$122,$74,$142,"5,336","5,040,000","18,000"


#### Q3

In [29]:
test_input_csv = list('Input_OPT_budget' = 900000,
'Input_SDL_budget' = 350000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,250",$564,$0,$0,$145,$339,$203,"16,038","45,010,000","65,000"
EAST,$900,$408,$0,$0,$71,$296,$124,"10,223","40,000,000","50,000"
WEST,$350,$155,$0,$0,$74,$42,$79,"5,816","5,010,000","15,000"


In [30]:
test_input_csv = list('Input_OPT_budget' = 920000,
'Input_SDL_budget' = 360000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,280",$588,$0,$0,$149,$340,$203,"16,109","50,040,000","67,000"
EAST,$920,$428,$0,$0,$71,$296,$124,"10,283","45,000,000","51,000"
WEST,$360,$159,$0,$0,$78,$43,$79,"5,827","5,040,000","16,000"


In [31]:
test_input_csv = list('Input_OPT_budget' = 1620000,
'Input_SDL_budget' = 620000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,240","$1,002",$0,$0,$254,$625,$359,"16,803","50,040,000","71,000"
EAST,"$1,620",$720,$0,$0,$132,$550,$218,"10,797","45,000,000","53,000"
WEST,$620,$282,$0,$0,$122,$74,$142,"6,006","5,040,000","18,000"


#### Q4

In [32]:
test_input_csv = list('Input_OPT_budget' = 900000,
'Input_SDL_budget' = 350000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,250",$564,$0,$0,$145,$339,$203,"14,818","45,010,000","65,000"
EAST,$900,$408,$0,$0,$71,$296,$124,"9,917","40,000,000","50,000"
WEST,$350,$155,$0,$0,$74,$42,$79,"4,901","5,010,000","15,000"


In [33]:
test_input_csv = list('Input_OPT_budget' = 920000,
'Input_SDL_budget' = 360000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,280",$588,$0,$0,$149,$340,$203,"14,889","50,040,000","67,000"
EAST,$920,$428,$0,$0,$71,$296,$124,"9,977","45,000,000","51,000"
WEST,$360,$159,$0,$0,$78,$43,$79,"4,911","5,040,000","16,000"


In [34]:
test_input_csv = list('Input_OPT_budget' = 1620000,
'Input_SDL_budget' = 620000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,240","$1,002",$0,$0,$254,$625,$359,"15,583","50,040,000","71,000"
EAST,"$1,620",$720,$0,$0,$132,$550,$218,"10,492","45,000,000","53,000"
WEST,$620,$282,$0,$0,$122,$74,$142,"5,091","5,040,000","18,000"


### DRTV OFF

#### Q1

In [35]:
test_input_csv = list('Input_OPT_budget' = 1100000,
'Input_SDL_budget' = 390000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,490",$546,$0,$260,$143,$338,$202,"16,110","45,010,000","65,000"
EAST,"$1,100",$392,$0,$217,$71,$296,$124,"10,804","40,000,000","50,000"
WEST,$390,$154,$0,$44,$72,$42,$78,"5,306","5,010,000","15,000"


In [36]:
test_input_csv = list('Input_OPT_budget' = 1300000,
'Input_SDL_budget' = 400000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,700",$651,$0,$359,$147,$339,$204,"16,707","50,040,000","67,000"
EAST,"$1,300",$494,$0,$314,$71,$296,$124,"11,391","45,000,000","51,000"
WEST,$400,$157,$0,$45,$76,$42,$80,"5,316","5,040,000","16,000"


In [37]:
test_input_csv = list('Input_OPT_budget' = 2000000,
'Input_SDL_budget' = 700000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,700","$1,002",$0,$467,$254,$625,$352,"18,079","50,040,000","71,000"
EAST,"$2,000",$720,$0,$388,$132,$550,$210,"12,569","45,000,000","53,000"
WEST,$700,$283,$0,$79,$122,$75,$142,"5,510","5,040,000","18,000"


#### Q2

In [38]:
test_input_csv = list('Input_OPT_budget' = 1100000,
'Input_SDL_budget' = 390000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,490",$546,$0,$260,$143,$338,$202,"16,508","45,010,000","65,000"
EAST,"$1,100",$392,$0,$217,$71,$296,$124,"11,361","40,000,000","50,000"
WEST,$390,$154,$0,$44,$72,$42,$78,"5,147","5,010,000","15,000"


In [39]:
test_input_csv = list('Input_OPT_budget' = 1300000,
'Input_SDL_budget' = 400000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,700",$651,$0,$359,$147,$339,$204,"17,105","50,040,000","67,000"
EAST,"$1,300",$494,$0,$314,$71,$296,$124,"11,948","45,000,000","51,000"
WEST,$400,$157,$0,$45,$76,$42,$80,"5,157","5,040,000","16,000"


In [40]:
test_input_csv = list('Input_OPT_budget' = 2000000,
'Input_SDL_budget' = 700000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,700","$1,002",$0,$467,$254,$625,$352,"18,477","50,040,000","71,000"
EAST,"$2,000",$720,$0,$388,$132,$550,$210,"13,126","45,000,000","53,000"
WEST,$700,$283,$0,$79,$122,$75,$142,"5,351","5,040,000","18,000"


#### Q3

In [41]:
test_input_csv = list('Input_OPT_budget' = 1100000,
'Input_SDL_budget' = 390000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,490",$546,$0,$260,$143,$338,$202,"16,385","45,010,000","65,000"
EAST,"$1,100",$392,$0,$217,$71,$296,$124,"10,568","40,000,000","50,000"
WEST,$390,$154,$0,$44,$72,$42,$78,"5,816","5,010,000","15,000"


In [42]:
test_input_csv = list('Input_OPT_budget' = 1300000,
'Input_SDL_budget' = 400000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,700",$651,$0,$359,$147,$339,$204,"16,982","50,040,000","67,000"
EAST,"$1,300",$494,$0,$314,$71,$296,$124,"11,155","45,000,000","51,000"
WEST,$400,$157,$0,$45,$76,$42,$80,"5,827","5,040,000","16,000"


In [43]:
test_input_csv = list('Input_OPT_budget' = 2000000,
'Input_SDL_budget' = 700000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,700","$1,002",$0,$467,$254,$625,$352,"18,354","50,040,000","71,000"
EAST,"$2,000",$720,$0,$388,$132,$550,$210,"12,334","45,000,000","53,000"
WEST,$700,$283,$0,$79,$122,$75,$142,"6,020","5,040,000","18,000"


#### Q4

In [44]:
test_input_csv = list('Input_OPT_budget' = 1100000,
'Input_SDL_budget' = 390000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,490",$546,$0,$260,$143,$338,$202,"15,164","45,010,000","65,000"
EAST,"$1,100",$392,$0,$217,$71,$296,$124,"10,263","40,000,000","50,000"
WEST,$390,$154,$0,$44,$72,$42,$78,"4,901","5,010,000","15,000"


In [45]:
test_input_csv = list('Input_OPT_budget' = 1300000,
'Input_SDL_budget' = 400000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,700",$651,$0,$359,$147,$339,$204,"15,762","50,040,000","67,000"
EAST,"$1,300",$494,$0,$314,$71,$296,$124,"10,850","45,000,000","51,000"
WEST,$400,$157,$0,$45,$76,$42,$80,"4,912","5,040,000","16,000"


In [46]:
test_input_csv = list('Input_OPT_budget' = 2000000,
'Input_SDL_budget' = 700000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 1,
'flight_flag_radio_SDL' = 1, 
'flight_flag_DRTV_OPT' = 0,
'flight_flag_DRTV_SDL' = 0,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,700","$1,002",$0,$467,$254,$625,$352,"17,133","50,040,000","71,000"
EAST,"$2,000",$720,$0,$388,$132,$550,$210,"12,028","45,000,000","53,000"
WEST,$700,$283,$0,$79,$122,$75,$142,"5,105","5,040,000","18,000"


### Radio OFF

#### Q1

In [47]:
test_input_csv = list('Input_OPT_budget' = 1100000,
'Input_SDL_budget' = 470000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,570",$546,$344,$0,$139,$339,$202,"15,949","45,010,000","65,000"
EAST,"$1,100",$391,$217,$0,$71,$296,$124,"10,549","40,000,000","50,000"
WEST,$470,$155,$127,$0,$68,$42,$78,"5,400","5,010,000","15,000"


In [48]:
test_input_csv = list('Input_OPT_budget' = 1300000,
'Input_SDL_budget' = 480000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,780",$640,$457,$0,$141,$339,$203,"16,249","50,040,000","67,000"
EAST,"$1,300",$482,$326,$0,$71,$296,$124,"10,826","45,000,000","51,000"
WEST,$480,$157,$131,$0,$70,$43,$79,"5,423","5,040,000","16,000"


In [49]:
test_input_csv = list('Input_OPT_budget' = 2000000,
'Input_SDL_budget' = 840000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q1',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q1],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,840","$1,000",$614,$0,$254,$618,$354,"17,684","50,040,000","71,000"
EAST,"$2,000",$720,$383,$0,$132,$550,$215,"11,595","45,000,000","53,000"
WEST,$840,$280,$231,$0,$122,$67,$140,"6,090","5,040,000","18,000"


#### Q2

In [50]:
test_input_csv = list('Input_OPT_budget' = 1100000,
'Input_SDL_budget' = 470000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,570",$546,$344,$0,$139,$339,$202,"16,347","45,010,000","65,000"
EAST,"$1,100",$391,$217,$0,$71,$296,$124,"11,106","40,000,000","50,000"
WEST,$470,$155,$127,$0,$68,$42,$78,"5,241","5,010,000","15,000"


In [51]:
test_input_csv = list('Input_OPT_budget' = 1300000,
'Input_SDL_budget' = 480000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,780",$640,$457,$0,$141,$339,$203,"16,647","50,040,000","67,000"
EAST,"$1,300",$482,$326,$0,$71,$296,$124,"11,383","45,000,000","51,000"
WEST,$480,$157,$131,$0,$70,$43,$79,"5,264","5,040,000","16,000"


In [52]:
test_input_csv = list('Input_OPT_budget' = 2000000,
'Input_SDL_budget' = 840000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q2',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q2],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,840","$1,000",$614,$0,$254,$618,$354,"18,083","50,040,000","71,000"
EAST,"$2,000",$720,$383,$0,$132,$550,$215,"12,152","45,000,000","53,000"
WEST,$840,$280,$231,$0,$122,$67,$140,"5,930","5,040,000","18,000"


#### Q3

In [53]:
test_input_csv = list('Input_OPT_budget' = 1100000,
'Input_SDL_budget' = 470000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,570",$546,$344,$0,$139,$339,$202,"16,224","45,010,000","65,000"
EAST,"$1,100",$391,$217,$0,$71,$296,$124,"10,313","40,000,000","50,000"
WEST,$470,$155,$127,$0,$68,$42,$78,"5,910","5,010,000","15,000"


In [54]:
test_input_csv = list('Input_OPT_budget' = 1300000,
'Input_SDL_budget' = 480000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,780",$640,$457,$0,$141,$339,$203,"16,524","50,040,000","67,000"
EAST,"$1,300",$482,$326,$0,$71,$296,$124,"10,590","45,000,000","51,000"
WEST,$480,$157,$131,$0,$70,$43,$79,"5,933","5,040,000","16,000"


In [55]:
test_input_csv = list('Input_OPT_budget' = 2000000,
'Input_SDL_budget' = 840000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q3',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q3],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,840","$1,000",$614,$0,$254,$618,$354,"17,959","50,040,000","71,000"
EAST,"$2,000",$720,$383,$0,$132,$550,$215,"11,359","45,000,000","53,000"
WEST,$840,$280,$231,$0,$122,$67,$140,"6,600","5,040,000","18,000"


#### Q4

In [56]:
test_input_csv = list('Input_OPT_budget' = 1100000,
'Input_SDL_budget' = 470000,
'Input_OPT_crosschannel_imp' = 40000000,
'Input_SDL_crosschannel_imp' = 5010000,
'Input_OPT_email_sent' = 50000,
'Input_SDL_email_sent' = 15000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,570",$546,$344,$0,$139,$339,$202,"15,003","45,010,000","65,000"
EAST,"$1,100",$391,$217,$0,$71,$296,$124,"10,008","40,000,000","50,000"
WEST,$470,$155,$127,$0,$68,$42,$78,"4,995","5,010,000","15,000"


In [57]:
test_input_csv = list('Input_OPT_budget' = 1300000,
'Input_SDL_budget' = 480000,
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 51000,
'Input_SDL_email_sent' = 16000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$1,780",$640,$457,$0,$141,$339,$203,"15,303","50,040,000","67,000"
EAST,"$1,300",$482,$326,$0,$71,$296,$124,"10,285","45,000,000","51,000"
WEST,$480,$157,$131,$0,$70,$43,$79,"5,018","5,040,000","16,000"


In [58]:
test_input_csv = list('Input_OPT_budget' = 2000000,
'Input_SDL_budget' = 840000,                                  #changed
'Input_OPT_crosschannel_imp' = 45000000,
'Input_SDL_crosschannel_imp' = 5040000,
'Input_OPT_email_sent' = 53000,
'Input_SDL_email_sent' = 18000,                       
'input_period' = 'Q4',
'flight_flag_radio_OPT' = 0,
'flight_flag_radio_SDL' = 0, 
'flight_flag_DRTV_OPT' = 1,
'flight_flag_DRTV_SDL' = 1,
'output_type' = 'csv'  
)

run_b2b_app(test_input_csv)

Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Warning message:
"Column `dma` joining factor and character vector, coercing into character vector"Adding missing grouping variables: `qy`
Warning message:
"Column `dma` joining character vector and factor, coercing into character vector"Warning message:
"Column `Footprint` joining character vector and factor, coercing into character vector"

Footprint,Total Spend $(000) [Q4],Digital,DRTV,Radio,Direct Mail,Search,Social,Gross Adds,Cross Channel Impressions,Email Volume
USA,"$2,840","$1,000",$614,$0,$254,$618,$354,"16,739","50,040,000","71,000"
EAST,"$2,000",$720,$383,$0,$132,$550,$215,"11,054","45,000,000","53,000"
WEST,$840,$280,$231,$0,$122,$67,$140,"5,685","5,040,000","18,000"


### Sai - B2C Test case generations

In [25]:
input_df <- data.frame('key' = names(test_input_csv),'value' = as.character(unlist(test_input_csv)))
input_df

key,value
Input_OPT_budget,19500000
Input_SDL_budget,11100000
constraint,yes
input_period,Q4
n_flight_weeks,13
OPT_DM_budget,5400000
SDL_DM_budget,3900000
output_type,csv


In [26]:
ncol_final = max(ncol(input_df),ncol(test_res_csv))
ncol_input = ncol(input_df)
nrow_input = nrow(input_df)
ncol_diff = ncol_final - ncol_input


final_inputs_df = data.frame(matrix(nrow = 0,ncol = ncol_final))

##Create dummy columns for the input df to make it have the same columns as test_res_csv
dummy_df = data.frame(matrix(rep(NA,ncol_diff*nrow_input),nrow = nrow_input,ncol = ncol_diff))



input_df = cbind(input_df,dummy_df)

colnames(input_df) <- paste0('X',c(1:ncol_final))

test_res_csv = rbind(colnames(test_res_csv),test_res_csv)
colnames(test_res_csv) = paste0('X',c(1:ncol_final))

final_inputs_df = rbind(input_df,test_res_csv)

final_inputs_df

X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
Input_OPT_budget,19500000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Input_SDL_budget,11100000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
constraint,yes,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
input_period,Q4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
n_flight_weeks,13,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
OPT_DM_budget,5400000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
SDL_DM_budget,3900000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
output_type,csv,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Footprint,Spend $(000) [Q4],TV,Direct Mail [Constrained],Paid Search Brand,Paid Search Unbranded,Display,Radio,Video,Social,Print [13 Weeks],Audio
"USA [TGA :202,980]","$39,900","$13,018","$9,300","$4,432","$3,875","$3,712","$2,147","$1,951","$1,106",$257,$101


In [27]:
write.table(final_inputs_df,'test_package.csv',row.names = FALSE,col.names = FALSE,sep = ',',na = '')

#### Test Case generation

In [8]:
generate_test_cases <- function(input,output){
    input_df <- data.frame('key' = names(input),'value' = as.character(unlist(input)))
    ncol_final = max(ncol(input_df),ncol(output))
    ncol_input = ncol(input_df)
    nrow_input = nrow(input_df)
    ncol_diff = ncol_final - ncol_input


    final_inputs_df = data.frame(matrix(nrow = 0,ncol = ncol_final),stringsAsFactors = F)

    ##Create dummy columns for the input df to make it have the same columns as test_res_csv
    dummy_df = data.frame(matrix(rep(NA,ncol_diff*nrow_input),nrow = nrow_input,ncol = ncol_diff),stringsAsFactors = F)



    input_df = cbind(input_df,dummy_df)

    colnames(input_df) <- paste0('X',c(1:ncol_final))
    #output <- data.frame(output,stringsAsFactors = F)
    output = rbind(colnames(output),output)
    colnames(output) = paste0('X',c(1:ncol_final))

    final_inputs_df = rbind(input_df,output)
    
    return(final_inputs_df)
}

In [4]:
gen_b2c_test_cases <- function(opt_budget_vec,sdl_budget_vec,constraint){
    
    output_list = list()
    ##Generate the combinations of both vectors
    cases_df <- crossing(opt_budget_vec,sdl_budget_vec)
   
    
    
    idx <- c(1:nrow(cases_df))
    
    opt_dm_budget <- sample(c(3300000,6100000),size = 1, replace = T)
    sdl_dm_budget <- sample(c(2400000,4400000),size = 1,replace = T)
    
    for(id in idx){
        budget_vec = cases_df %>% slice(id) %>% unlist(use.names = FALSE)
        quarter <- paste0('Q',sample(1:4,size = 1,replace = T))
        print_flag_OPT = T
        print_flag_SDL = T
        ##If the extracted budget is lower bound for respective footprint, turn print off
        if(budget_vec[1]%in%c(16600000,19900000)){
            print_flag_OPT = F
        }
        if(budget_vec[2]%in%c(8050000,10400000)){
            print_flag_SDL = F
        }
        cat(budget_vec)
        cat('\n')
        cat(sprintf('OPT budget: %s\n',format_money(budget_vec[1],0)))
        cat(sprintf('SDL budget: %s\n',format_money(budget_vec[2],0)))
        cat(sprintf('Quarter selected: %s\n',quarter))
        
        
        test_input_case = list('Input_OPT_budget' = budget_vec[1],
                                'Input_SDL_budget' = budget_vec[2],
                                'constraint' = constraint,
                                'input_period' = quarter,
                                'flight_flag_OPT' = print_flag_OPT,
                                'flight_flag_SDL' = print_flag_SDL,                       
                                'OPT_DM_budget' = opt_dm_budget,
                                'SDL_DM_budget' = sdl_dm_budget,
                                'output_type' = 'csv' 
                                )
        
        gen_output <- run_mmm_app(test_input_case)
        
        test_case <- generate_test_cases(test_input_case,gen_output)
        
        output_name <- paste0('case',idx)
        temp_list <- list(test_case)
        names(temp_list) <- paste('case',id,sep = '')
        output_list <- append(output_list,temp_list)
        
    }
    return(output_list)
}

In [9]:
opt_budget_vec_no_dm <- c(16900000,30800000,16600000)
sdl_budget_vec_no_dm <- c(8900000,14850000,8050000)

In [10]:
opt_budget_vec_dm <- c(20200000,36900000,19900000)
sdl_budget_vec_dm <- c(11300000,10400000,19250000)

In [11]:
res_no_dm <- gen_b2c_test_cases(opt_budget_vec_no_dm,sdl_budget_vec_no_dm,'yes')

16600000 8050000
OPT budget: $16,600,000
SDL budget: $8,050,000
Quarter selected: Q3


Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

16600000 8900000
OPT budget: $16,600,000
SDL budget: $8,900,000
Quarter selected: Q1


Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

16600000 14850000
OPT budget: $16,600,000
SDL budget: $14,850,000
Quarter selected: Q1


...

Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

30800000 8900000
OPT budget: $30,800,000
SDL budget: $8,900,000
Quarter selected: Q2


Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

30800000 14850000
OPT budget: $30,800,000
SDL budget: $14,850,000
Quarter selected: Q4


Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

In [12]:
res_dm <- gen_b2c_test_cases(opt_budget_vec_dm,sdl_budget_vec_dm,'no')

19900000 10400000
OPT budget: $19,900,000
SDL budget: $10,400,000
Quarter selected: Q2


Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

19900000 11300000
OPT budget: $19,900,000
SDL budget: $11,300,000
Quarter selected: Q2


Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

19900000 19250000
OPT budget: $19,900,000
SDL budget: $19,250,000
Quarter selected: Q1


...

Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

36900000 11300000
OPT budget: $36,900,000
SDL budget: $11,300,000
Quarter selected: Q1


Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

36900000 19250000
OPT budget: $36,900,000
SDL budget: $19,250,000
Quarter selected: Q1


Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Warning message:
“Column `dma` joining factor and character vector, coercing into character vector”Adding missing grouping variables: `qy`
Warning message:
“Column `dma` joining character vector and factor, coercing into character vector”

In [84]:
res_dm$case1

X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
Input_OPT_budget,19900000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Input_SDL_budget,10400000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
constraint,no,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
input_period,Q3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
flight_flag_OPT,FALSE,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
flight_flag_SDL,FALSE,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
OPT_DM_budget,3300000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
SDL_DM_budget,2400000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
output_type,csv,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Footprint,Spend $(000) [Q3],TV,Direct Mail [No Constraint],Display,Paid Search Unbranded,Paid Search Brand,Video,Radio,Social,Audio,Print


In [13]:
for(name in names(res_no_dm)){
    output_df <- res_no_dm[[name]]
    fname <- paste0('test','_no_dm_',name,'.csv')
    write.table(output_df,fname,row.names = F,na = '',sep = ',',col.names = F)
}

In [14]:
for(name in names(res_dm)){
    output_df <- res_dm[[name]]
    fname <- paste0('test','_dm_',name,'.csv')
    write.table(output_df,fname,row.names = F,na = '',sep = ',',col.names = F)
}

In [15]:
res_dm$case9

X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
Input_OPT_budget,36900000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Input_SDL_budget,19250000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
constraint,no,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
input_period,Q1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
flight_flag_OPT,TRUE,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
flight_flag_SDL,TRUE,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
OPT_DM_budget,3300000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
SDL_DM_budget,4400000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
output_type,csv,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Footprint,Spend $(000) [Q1],TV,Direct Mail [No Constraint],Display,Paid Search Unbranded,Paid Search Brand,Video,Radio,Social,Print,Audio
